### Requirements:
You have 2 sample datasets attached to this test:
- price.csv: records of product's history price changes in Sep 2018
- sales.csv: records of product's sales in Sep 2018

Use Python to calculate the total revenue of each product at each price. The solution must be applicable to similar datasets with large numbers of records

In [23]:
import pandas as pd
import numpy as np
sales = pd.read_csv('sales.csv')
prices = pd.read_csv('prices.csv')
sales.info()
prices.info()
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])
prices['updated_at'] = pd.to_datetime(prices['updated_at'])
sales = sales.sort_values('ordered_at')
prices = prices.sort_values('updated_at')
a = pd.merge_asof(sales, prices, left_on='ordered_at', right_on='updated_at', by='product_id', direction='nearest')
a['official_price'] = np.where(a['ordered_at'] >= a['updated_at'], a['new_price'], a['old_price'])
a['revenue'] = a['official_price'] * a['quantity_ordered']
final_a = a.groupby(['product_id','official_price'],as_index=False)['revenue'].sum()
print(final_a)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        175 non-null    int64 
 1   ordered_at        175 non-null    object
 2   quantity_ordered  175 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  14 non-null     int64 
 1   old_price   14 non-null     int64 
 2   new_price   14 non-null     int64 
 3   updated_at  14 non-null     object
dtypes: int64(3), object(1)
memory usage: 576.0+ bytes
    product_id  official_price  revenue
0           64          239000   956000
1      3954203           57500    57500
2      3954203           60000   180000
3      3954203           64000   640000
4      3998909     

In [24]:
#Convert ordered_at, updated_at to datetime object 
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])
prices['updated_at'] = pd.to_datetime(prices['updated_at'])

In [25]:
sales = sales.sort_values('ordered_at')
prices = prices.sort_values('updated_at')

In [26]:
#Find the "nearest" update timestamp of each order
a = pd.merge_asof(sales, prices, left_on='ordered_at', right_on='updated_at', by='product_id', direction='nearest')

In [27]:
#Compare ordered_at and "nearest" updated_at of each order to find the correct price
a['official_price'] = np.where(a['ordered_at'] >= a['updated_at'], a['new_price'], a['old_price'])
a.head(10)

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at,official_price
0,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00,60000
1,4085861,2018-09-11 06:26:00,1,60000,53500,2018-09-11 08:51:00,60000
2,4085861,2018-09-11 06:53:00,1,60000,53500,2018-09-11 08:51:00,60000
3,4085861,2018-09-11 08:24:00,1,60000,53500,2018-09-11 08:51:00,60000
4,4085861,2018-09-11 09:30:00,1,60000,53500,2018-09-11 08:51:00,53500
5,4085861,2018-09-11 11:06:00,1,60000,53500,2018-09-11 08:51:00,53500
6,3954203,2018-09-11 11:11:00,1,60000,64000,2018-09-11 11:54:00,60000
7,3954203,2018-09-11 11:11:00,1,60000,64000,2018-09-11 11:54:00,60000
8,4085861,2018-09-11 11:34:00,1,60000,53500,2018-09-11 08:51:00,53500
9,4085861,2018-09-11 11:47:00,2,60000,53500,2018-09-11 08:51:00,53500


In [28]:
#Calculate total revenue for each product
a['revenue'] = a['official_price'] * a['quantity_ordered']
final_a = a.groupby(['product_id','official_price'],as_index=False)['revenue'].sum()

In [29]:
print(a.groupby('product_id').agg({'revenue':'sum'}))

            revenue
product_id         
64           956000
3954203      877500
3998909      280500
4085861     8247500
